In [123]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import plotly.express as px  # Be sure to import express
import us

In [116]:
# Read and inspect the dataset
# Confirmed Cases
df_confirmed = pd.read_csv('time_series_covid19_confirmed_US.csv')
df_confirmed.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,9/9/20,9/10/20,9/11/20,9/12/20,9/13/20,9/14/20,9/15/20,9/16/20,9/17/20,9/18/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1398,1413,1420,1432,1442,1447,1463,1619,1624,1664
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,4639,4678,4722,4752,4781,4800,4812,5003,5021,5033
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,618,618,618,620,622,626,629,809,809,824
3,84001007,US,USA,840,1007.0,Bibb,Alabama,US,32.996421,-87.125115,...,564,566,574,576,578,581,580,612,617,619
4,84001009,US,USA,840,1009.0,Blount,Alabama,US,33.982109,-86.567906,...,1082,1099,1109,1114,1121,1128,1139,1487,1504,1527


In [117]:
# Read and inspect the dataset
# Deaths
df_deaths = pd.read_csv('time_series_covid19_deaths_US.csv')
df_deaths.tail(100)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,9/9/20,9/10/20,9/11/20,9/12/20,9/13/20,9/14/20,9/15/20,9/16/20,9/17/20,9/18/20
3240,84054109,US,USA,840,54109.0,Wyoming,West Virginia,US,37.612327,-81.545111,...,3,4,3,3,3,3,3,3,3,3
3241,84055001,US,USA,840,55001.0,Adams,Wisconsin,US,43.969747,-89.767828,...,3,3,3,3,3,3,3,3,3,3
3242,84055003,US,USA,840,55003.0,Ashland,Wisconsin,US,46.319569,-90.678371,...,1,1,1,1,1,1,1,1,1,2
3243,84055005,US,USA,840,55005.0,Barron,Wisconsin,US,45.423727,-91.848336,...,3,4,4,4,4,4,4,4,4,4
3244,84055007,US,USA,840,55007.0,Bayfield,Wisconsin,US,46.528688,-91.197167,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3335,84056039,US,USA,840,56039.0,Teton,Wyoming,US,43.935225,-110.589080,...,1,1,1,1,1,1,1,1,1,1
3336,84056041,US,USA,840,56041.0,Uinta,Wyoming,US,41.287818,-110.547578,...,2,2,2,2,2,2,2,2,2,2
3337,84090056,US,USA,840,90056.0,Unassigned,Wyoming,US,0.000000,0.000000,...,0,0,0,0,0,0,0,4,0,0
3338,84056043,US,USA,840,56043.0,Washakie,Wyoming,US,43.904516,-107.680187,...,6,6,6,6,6,6,6,6,6,6


In [118]:
# Transform the data from wide to long
df_deaths = pd.melt(df_deaths.drop(columns=['iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 
                    'Country_Region','Combined_Key']),
            id_vars=['UID', 'Population','Province_State'],var_name='date', value_name='deaths')

df_confirmed = pd.melt(df_confirmed.drop(columns=['iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 
                    'Country_Region','Combined_Key']),
            id_vars=['UID', 'Province_State'],var_name='date', value_name='confirmed')

df = df_deaths.merge(df_confirmed[['UID','date','confirmed']], on = ['UID', 'date'])

# Group by State
df = df[['date','Province_State','Population','deaths','confirmed']].groupby(['date','Province_State']).sum().reset_index()

# Filter out 'Diamond Princess', 'Grand Princess'
df = df[~df.Province_State.isin(['Diamond Princess', 'Grand Princess'])]

# Get the state code for each US State
df['State_Code'] = [us.states.lookup(x).abbr for x in df['Province_State']]

In [119]:
# Filter to the most current date (for checking)
df = df[df['date']=='9/18/20']

In [120]:
fig = px.choropleth(df,  # Input Pandas DataFrame
                    locations="State_Code",  # DataFrame column with locations
                    color="confirmed",  # DataFrame column with color values
                    hover_name="Province_State", # DataFrame column hover info
                    locationmode = 'USA-states') # Set to plot as US States

fig.update_layout(
    title_text = 'Confirmed Cases', # Create a Title
    geo_scope='usa',  # Plot only the USA instead of globe
)

fig.show()  # Output the plot to the screen

In [121]:
fig = px.scatter_geo(df, locations="State_Code",
                     hover_name="Province_State", size="confirmed")
fig.show()  # Output the plot to the screen